<p align="right">
<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/main/Dataflow/media/BYU.svg"
width="500" height="" style="display: block; margin: 0 auto" />
</p>


<h1>
<summary>
<h1 align="center">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>$\color{midnightBlue}{\underline{\textbf{Arithmetic}}}$<bol><p align="right">
</summary>
</h1>

## **Setup**
Click the Play Button. No need to expand. Do not touch this. 
<details><summary>Packages to be downloaded</summary>
Things it needs to install:  <br>

 *    Verilator- the simulator  <br>
 *    PyGithub - Helps with simulation    <br>
 *    VCD - Generates waveforms from `.vcd` files  <br>
 *    Widgets - Each widget must first be generated then will be loaded when played
</details>

In [ ]:
#@title Install VCD { form-width: "1795px" }
%%bash
git clone https://github.com/yne/vcd.git
make -C /content/vcd
cd /content/vcd
make install

In [ ]:
#@title Import Files from Repo
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests
def import_text(text):
  url = 'https://raw.githubusercontent.com/byuccl/digital_design_colab/main/Arithmetic/files/%s' % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)
def import_text_bin(text):
  url = 'https://raw.githubusercontent.com/byuccl/digital_design_colab/main/bin/%s' % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)
# import_text("tb_dataflow_sv.cpp")
# import_text("dataflow_sv3.cpp")
# import_text("tb_func2.cpp")
import_text("quiz_dict01.py")
import_text("tt_widget.py")
import_text("frquestion.py")

import tt_widget as tt
import quiz_dict01 as quiz_bot 
from frquestion import *
def quiz_func(a):
  quiz_bot.quiz_func(a)

!git clone https://github.com/byuccl/digital_design_colab.git

In [ ]:
#@title Anon Simulation Widget { form-width: "50px" }
import os
import re
import sys
import subprocess
from collections import defaultdict, namedtuple
import subprocess
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
!mkdir -p /content/tmp_code
def vcd():
  !vcd < /content/tmp_code/logs/vlt_dump.vcd > /content/vcd.txt
  !sed -i '1,10d'  /content/vcd.txt 
  !cat /content/vcd.txt


VerilogIO = namedtuple("VerilogIO", "dir nam wid")
'''
        Using a namedtuple (sort of like a C-struct) to hold info on each Verilog IO.
        This is very similar to using a class with only data members.
          The advantages are: (1) the fields are not change-able after creation
          (which is what I want), and (2) probably more efficient than using a class.
        This is used by both generate() and processVmoduleHFile() so it is declared in global scope.
'''


def getModuleName(textSourceCode):
  match_name = re.search(r"module\s{1,}\w{1,}\s{0,}#{0,}\s{0,}\(",textSourceCode.value)
  if match_name is None:
      return None
  mname = match_name.group().split(' ')[1]
  mname = mname.split('(')[0]
  mname = mname.split('#')[0]
  return mname

def writeJson(d, fname='/tmp/json.json', ind=2):
    ''' Write an object to a file with indenting '''
    import json
    with open(fname, encoding='utf-8', mode='w') as f:
        json.dump(d, f, indent=ind)

def cd(folder):
    ''' My wrapper for os.chdir() so I can print it desired. '''
    if folder:
        os.chdir(folder)
    #print("##### Changing to " + os.getcwd() + "#####")

def getNextLine(f):
    ''' Read file f until you get a non-comment and a non-blank link.
        The returned line will be stripped of whitespace (including newline)
    '''
    while True:
        myLine = f.readline().strip()
        if not myLine.startswith('//') and len(myLine) > 0:
            break
    return myLine

def processSTMHeader(tBench):
    '''
        Read STM file and pull out the clock definition and the list of inputs.
        Note that for combinational circuits we are expecting a clock line
          consisting of 'comb' (which is ignored)
    '''
    with open(tBench, encoding='utf-8', mode='r') as f:
        stmClock = None
        tmp = getNextLine(f)
        if tmp != 'comb':
            stmClock = tmp
        inputs = getNextLine(f).split()

    # Verify that a signal doesn't appear twice in the list (this is a user-created file after all)
    for elmt in inputs:
        assert inputs.count(elmt) == 1, "Cannot have duplicate names in STM file input list. ({} appears more than once)".format(elmt)
    return (stmClock, inputs)

def processVmoduleHFile(fname, stmClock, stmInputs):
    '''
        Process the verilator-produced Vmodule.h file and pull out
        the IO definitions.  Pack them into the verilogIOs structure,
        which is a list of lists consisting of [dir, name, size].
        Along the way see if you can match any of the signals as
        corresponding to the clock defined in the stimulus file.
        Finally, make sure that all verilog signals are in the stm file
          and the other way around.
    '''

    verilogIOs = []
    with open(fname, encoding='utf-8', mode='r') as f:
        verilogClock = None
        for line in f:
            line = line.strip()
            if line.startswith('// LOCAL VARIABLES'): # Can skip everything after this line
                break
            match = re.match(r'^VL_(IN|OUT|SIG)(\d*)\(([^,]*),(\d+),(\d+)', line)
            if match is not None:
                varDir, varName, varSize = [match.groups()[0], match.groups()[2], int(match.groups()[3]) - int(match.groups()[4]) +1]
                verilogIOs.append(VerilogIO(varDir, varName, varSize))
                # If input, check that it is in STM file
                if varName == stmClock:
                    verilogClock = varName   # Found matching clock names between STM and Verilog header

    # Check to make sure that every stm input is in the verilog file
    #   and the other way around.
    for si in stmInputs:
        assert si in [f.nam for f in verilogIOs if f.dir=='IN'], \
            "ERROR: input '{}' in STM file but not in Verilog design".format(si)
    for vi in [f.nam for f in verilogIOs if f.dir=='IN']:
        assert vi == stmClock or vi in stmInputs, \
            "ERROR: input '{}' in Verilog design but not in STM file".format(vi)

    # Make sure that if a clock was in the STM file, it was found in the Verilog
    if stmClock != None:
        assert verilogClock != None, "There was a clock defined in the STM file but no corresponding clock in the Verilog file."
    return (verilogClock, verilogIOs)
from string import Template
def generate(designFolder, verilogFile, tBenchFile,
             draw = True, wall = True, compress = True, Hex = True, verbose = False,
             pythonPath = '', processClockWaveform = True,
             hscale = 1, nameOrder = ''):
    """
    Generate a C++ object of verilog/systemverilog code using verilator, toggle inputs on said C++ object,
    and portray resulting waveforms of the signals. Comments in the stm file are supported and are indicated with //
    Comments must start at the beginning of the line and are expected to take up the whole line
    Keyword Arguements:
    designFolder -- str - relative path of the directory, where the .v and .stm files are
    verilogFile -- str - the top level HDL module
    tBenchFile -- str - the name of a .stm file of input stimulus.
    draw -- bool - whether or not to display the waveform
    wall -- bool whether or not to check for verilator warnings
    compress -- bool whether or not to compress the c++ object made by verilator. (Compression effectively
      eliminates the ability to view most internal signals  (deprecated)
    Hex -- bool - whether to display waveform values in hexadecimal
    verbose -- bool - control verbose mode
    pythonPath -- str - path to directory where MaverickSimulator project can be found. If left empty, the typical
                        PYTHONPATH/site-packages will be assumed
    processClockWaveform -- bool -  By default, will attempt to convert the clock waveform to one with arrows.
                                    Setting this to False will prevent this
    hscale -- int - Will adjust the width of the ticks on the waveform diagram. This arguement is passed directly
                    to executeSimulation()
    """
    clock = None    # The name of the clock to the design (if any)
    homeDir = os.getcwd()
    try:
        fileNameRoot = os.path.splitext(verilogFile)[0]
        if not pythonPath:
            dirs = sys.path
            for dir in dirs:
                if (dir.find('site-packages') != -1 ):
                    #TODO add check to make sure path is valid
                    pythonPath = dir + '/MaverickSimulator/src'
                    break
        # Adjust commandline args for verilator
        w = ' -Wall' if wall else ' '
        c = ' ' if compress else ' --public '
        wv = ' --trace ' if draw else '--trace'

        cd(designFolder)

        # Step 1: run the verilator program
        v = subprocess.Popen(r'verilator -cc'+ w + c + wv + verilogFile + " -CFLAGS '-std=c++11' --exe sim___.cpp",    #create verilator object and link to .cpp
                             stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)
        verbose = 0
        if verbose:
            print('Running verilator...')
            print('verilator -cc'+ w + c + wv + verilogFile + " -CFLAGS '-std=c++11' --exe sim___.cpp")
            output = v.stdout.read().strip()
            if len(output) > 0:
                print('Results of running verilator: ' + output)
            if output.find('%Error: Command Failed') != -1:
                return 1
                sys.exit("Exiting due to verilator errors")
        else:
            output = v.stdout.read().strip()
            if output.find('%Error: Command Failed') != -1:
                print(output)
                os.chdir(homeDir)
                return 1
                sys.exit("Exiting due to verilator errors")

        # Step 2: Parse STM and Verilog files
        # Open .stm file and get clock and list of inputs from the  first two uncommented lines
        stmClock, stmInputs = processSTMHeader(tBenchFile)

        # Read the IO definitions from the Vmodule.h file
        # Add inputs, outputs, and local signals to data structure verilogIOs
        verilogClock, verilogIOs = processVmoduleHFile('obj_dir/V'+fileNameRoot+'.h', stmClock, stmInputs)

        # If verilogClock has been set that means
        # it matched up with the one from the STM file. Otherwise, it did not.
        # The rest of the code will use clock to know this.
        if verilogClock != None:
            clock = verilogClock

        # Step 3:Write the Testbench__lh file
        ios = ''
        sg = ''
        # Build list of IO print stmts for printIOs()
        for nam, wid in [(f.nam, f.wid) for f in verilogIOs if f.dir!='SIG']:
            print(str(nam) + str(wid))
            ios += '    std::cout << " {0} is " << std::bitset<{1}>(top->{0}) <<  std::endl;\n'.format(nam, wid)
        print(ios)
        # If needed, build list of IO print stmts for printInternalOs()
        if not compress:
            for nam, wid in [(f.nam, f.wid) for f in verilogIOs if f.dir=='SIG']:
                varName = nam.replace('v__DOT__', '').replace('__DOT__', ' ')
                sg += '   std::cout << "{} is " << std::bitset<{}>(top->{}) << std::endl;\n'.format(varName, wid, nam)
        # Now actually write the testBench file into the design directory
        with open('TestBench___.h', encoding='utf-8', mode='w') as tb: #header file for interacting with verilator object
            with open("/content/tmp_code/TBtemplate.txt", 'r') as tbTemplate:
                t = Template(tbTemplate.read())
                t = t.substitute(
                    MODULE=fileNameRoot,
                    CLKDEF = '\n#define CLK top->'+clock if clock else '',
                    IOPRINTS = ios,
                    INTERNALIOPRINTS = sg
                    )
                tb.write(t)

        # Step 4: write the main simulation file
        # Collect IO print code snippets (using simTemplate2.txt as pattern)
        ios = ''
        for inputName in stmInputs:
            for inputWidth in [f.wid for f in verilogIOs if f.nam==inputName]:
                with open("/content/tmp_code/simTemplate2.txt", 'r') as simTemplate2:
                    t = Template(simTemplate2.read())
                    t = t.substitute(SIGNAME = inputName, SIGWIDTH = inputWidth)
                    ios += t

        # Now write the main simulation file for compilation (using simTemplate.txt as pattern)
        with open('sim___.cpp', encoding='utf-8', mode='w') as sim:
            with open("/content/tmp_code/simTemplate.txt",
                    encoding='utf-8', mode='r') as simTemplate:  #now write main.cpp file
                t = Template(simTemplate.read())
                t = t.substitute(
                            NUMINS = len(stmInputs),
                            READINPUTSCODE = ios,
                            HASCLOCK = (0 if clock is None else 1)
                            )
                sim.write(t)

        # Step 5: Compile the resulting program
        print("Compiling")
        if verbose:
            print('make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd obj_dir')
            v = subprocess.Popen(r'make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd obj_dir',
                                 stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)
            output = v.stdout.read().strip()
            if len(output) > 0:  print('   Results of compiling: ' + output)
        else:
            v = subprocess.Popen(r'make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd /content/tmp_code/obj_dir',    #call makefile to g++ compile appropriate files
                                 stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)

        output = v.stdout.read().strip()
        # Step 6: Execute it
        # if not nameOrder:
        #   names = [f.nam for f in verilogIOs if f.dir != 'SIG']
        #   writeJson(names, fname=fileNameRoot+'.nmo')
        #   nameOrder = fileNameRoot+'.nmo'
        # cd(homeDir)
        # ret = executeSimulation(designFolder, verilogFile, tBenchFile, draw,
        #                     compress, Hex, verbose, clock, nameOrder,
        #                     processClockWaveform, hscale, pythonPath)
        # if draw is False:
        #     return ret
        0
        subprocess.run(["/content/tmp_code/obj_dir/V" +fileNameRoot, "/content/tmp_code" + fileNameRoot + ".stm", "--trace" ])
        # !./tmp_code/obj_dir/VbehavLoadableReg /content/tmp_code/temp.stm --trace
      # !vcd < /content/logs/vlt_dump.vcd
        subprocess.run(["vcd", "<", "/content/logs/vlt_dump.vcd"])
    except Exception as e:
        os.chdir(homeDir)
        print(str(e))
def showWaveformClicked(self):
    sc = cellDict[self]
    assert sc is not None, "Internal error with cellDict"

    if submitCodes(sc.interpreterHomeDir, sc.textSourceCode, sc.textStimulus, sc.selectHexBin.value) == False:
        return
    srcFileName = getModuleName(sc.textSourceCode)
    arrow = sc.chkArrow.value
    hex = True if sc.selectHexBin.value is 'Hex' else False
    assert os.path.exists(sc.interpreterHomeDir+'/tmp_code/'+srcFileName+'.sv'), "Please submit the codes before showing waveform: {} {}".format(sc.interpreterHomeDir, srcFileName)
    generate(sc.interpreterHomeDir+'/tmp_code', srcFileName+'.sv', srcFileName+'.stm', compress=True, Hex=hex, verbose=True, processClockWaveform=arrow)
    if sc.showCode.value:
        print("------------------------------------------------------------------------------------------")
        print("Source code for the simulation above:")
        print("------------------------------------------------------------------------------------------")
        print(sc.textSourceCode.value)
        print("------------------------------------------------------------------------------------------")

    print("###########################################################################################")
    x = subprocess.run(["/content/tmp_code/obj_dir/V" +srcFileName, "/content/tmp_code/" + srcFileName + ".stm", "--trace" ], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    print(x.stderr + x.stdout)
    # x = subprocess.run(["vcd", "< /content/tmp_code/logs/vlt_dump.vcd" ], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    # print(str(x.stderr) + str(x.stdout))
    # # os.system("vcd < /content/tmp_code/logs/vlt_dump.vcd" )
    vcd()
class SimCell:
  def __init__(self, textSourceCode, textStimulus, chkArrow, selectHexBin, interpreterHomeDir, showCode, contents):
      self.textSourceCode = textSourceCode
      self.textStimulus = textStimulus
      self.chkArrow = chkArrow
      self.selectHexBin = selectHexBin
      self.interpreterHomeDir = "/content"
      self.showCode = showCode
      self.contents = contents
def readContents(contents):
    if contents is None:
        code = ''
        stim = ''
    else:
        print(contents)
        contents = "/content/" + contents 
        with open(contents + ".sv") as f:
            code = f.read()
        with open(contents + ".stm") as f:
            stim = f.read()
        # with open(JTEXT+'/pgms/' + contents[:-1] + "tm") as f:
        #     stim = f.read()
    return [code, stim]

cellDict = dict()

def CreateWidgets(contents, ht="125px", wid="500px", stht='125px', stwid='500px'):
    code, stim = readContents(contents)

    srcCodeWidget = widgets.Textarea(
                            disabled=False,
                            value = code,
                            layout=widgets.Layout(height=ht,width=wid),
                            )

    stimulusWidget = widgets.Textarea(
                            value=stim,
                            disabled=False,
                            layout=widgets.Layout(height=stht, width=stwid),
                            )

    selectHexBin = widgets.Dropdown(options=['Binary','Hex', 'Decimal'],
                                 value='Decimal',
                                 description='',
                                 disabled=False,
                                 )

    chkArrow = widgets.Checkbox(description='Show the clock arrow', value=True, indent=False)
    showCode = widgets.Checkbox(description='Show source code below waveform', value=False, indent=False)
    #butt_submit = widgets.Button(description='Save code')
    butt_show_waveform = widgets.Button(description='Run simulation')
    butt_refresh_contents = widgets.Button(description='Restore code')
    butt_save_code = widgets.Button(description = "Save code")
    return srcCodeWidget, stimulusWidget, selectHexBin, chkArrow, showCode, butt_show_waveform, butt_refresh_contents, butt_save_code

def submitCodes(interpreterHomeDir, textSourceCode, textStimulus, selectHexBin):
    print(selectHexBin)
    if textSourceCode.value is '' or textStimulus.value is '':
        print("The source code and stimulus cannot be empty! Please input again")
        print("###########################################################################################")
        return False
    else:
        with open(interpreterHomeDir + "/tmp_code/" + getModuleName(textSourceCode) + ".sv", 'w') as f:
            f.write(textSourceCode.value)
        with open(interpreterHomeDir + "/tmp_code/" + getModuleName(textSourceCode) + ".stm", 'w') as f:
            temp_string = textStimulus.value
            temp_string = temp_string + '\n' + '\n'
            skip_two_lines = 0
            cur_substring = ""
            last_break = ''
            binary = ['0', '1', '\n', ' ']
            hex = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'A', 'B', 'C', 'D', 'E', 'F', ' ', '\n']
            if selectHexBin == 'Hex':
                for i in textStimulus.value:
                    if i == '\n' and skip_two_lines !=2:
                      skip_two_lines+=1
                      last_break = '\n'
                      continue
                    if skip_two_lines < 2:
                      continue
                    if hex.count(i) == 0:
                        print("Invalid Hex Character, Removing...")
                        temp_string = temp_string.replace(i, '')
                        continue
                    if i != ' ' and i != '\n':
                      cur_substring = cur_substring + i
                    elif i == ' ' and cur_substring:
                      temp_string = temp_string.replace(last_break + cur_substring + i, last_break + str(int(cur_substring, 16)) + ' ')
                      last_break = ' '
                      cur_substring = ""
                    elif i == '\n' and cur_substring:
                      temp_string = temp_string.replace(last_break + cur_substring + i, last_break + str(int(cur_substring, 16)) + '\n')
                      last_break = '\n'
                      cur_substring = ""
                    elif i == ' ':
                      last_break = ' '
                    elif i == '\n':
                      last_break = '\n'
            elif selectHexBin == 'Binary':
                  for i in textStimulus.value:
                    if i == '\n' and skip_two_lines !=2:
                      skip_two_lines+=1
                      last_break = '\n'
                      continue
                    if skip_two_lines < 2:
                      continue
                    if binary.count(i) == 0:
                        print("Invalid Hex Character, Skipping...")
                        continue
                    if i != ' ' and i != '\n':
                      cur_substring = cur_substring + i
                    elif i == ' ' and cur_substring:
                      temp_string = temp_string.replace(last_break + cur_substring + i, last_break + str(int(cur_substring, 2)) + ' ')
                      last_break = ' '
                      cur_substring = ""
                    elif i == '\n' and cur_substring:
                      temp_string = temp_string.replace(last_break + cur_substring + i, last_break + str(int(cur_substring, 2)) + '\n')
                      last_break = '\n'
                      cur_substring = ""
                    elif i == ' ':
                      last_break = ' '
                    elif i == '\n':
                      last_break = '\n'
            if selectHexBin != "Decimal":
              print(f"Converted Values of STM \n")
              print(temp_string)
            f.write(temp_string[:-1])
    return True

def refreshContentsClicked(self):
    sc = cellDict[self]
    assert sc is not None, "Internal error with cellDict"
    code, stim = readContents(sc.contents)
    sc.textSourceCode.value = code
    sc.textStimulus.value = stim

def overwriteSavedCode(self):
    sc = cellDict[self]
    print(sc.contents)
    file_location = "/content/" + sc.contents
    with open(file_location + ".sv", "w") as f:
      f.write(sc.textSourceCode.value)
    with open(file_location+".stm", "w") as f:
      f.write(sc.textStimulus.value)

def createSimulationWorkSpace(initialContents = None, ht='125px', wid='500px', stht='125px', stwid='500px'):

    interpreterHomeDir = os.getcwd()
    code_file_path = interpreterHomeDir+'/tmp_code'
    if not os.path.exists(code_file_path):
        os.mkdir(code_file_path)

    # Create widgets for this part
    textSourceCode, textStimulus, selectHexBin, chkArrow, showCode, btnShowWaveform, btnRestoreCode, btnSaveCode = CreateWidgets(initialContents, ht, wid, stht, stwid)
    #print(textSourceCode.value)
    #print(textStimulus.value)

    # Register callbacks for when clicking the button
    btnShowWaveform.on_click(showWaveformClicked)
    btnRestoreCode.on_click(refreshContentsClicked)
    btnSaveCode.on_click(overwriteSavedCode)
    # Register this cell's info
    sc = SimCell(textSourceCode, textStimulus, chkArrow, selectHexBin, interpreterHomeDir, showCode, initialContents)
    cellDict[btnShowWaveform] = sc
    cellDict[btnRestoreCode] = sc
    cellDict[btnSaveCode] = sc


    # Draw cell widgets
    leftPart = widgets.VBox([
                        widgets.VBox([
                                widgets.Label('Source code area:'),
                                textSourceCode
                                ]),
                        widgets.VBox([
                                widgets.Label('Simulation code area:'),
                                textStimulus
                                ])
                        ])
    rightPart = widgets.VBox([
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label('Select Simulation Input Type'),
                                selectHexBin,
                                showCode,
                                #widgets.Label(''),
                                btnShowWaveform,
                                #widgets.Label(''),
                                btnSaveCode,
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                btnRestoreCode
                            ])

    # Show widgets
    display(widgets.HBox([leftPart,rightPart]))




In [ ]:
#@title Import Simulation Files
cd("/content/tmp_code")
import_text_bin("TBtemplate.txt")
import_text_bin("simTemplate.txt")
import_text_bin("simTemplate2.txt")
import_text("full_add.sv")
import_text("full_add.stm")
import_text("add_8.sv")
import_text("add_8.stm")
import_text("arithmetic_top.sv")
import_text("arithmetic_top.stm")
import_text("tb_full_add.cpp")
import_text("tb_add_8.cpp")
cd("/content")

In [ ]:
#@title Grid Creation
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider

# function of creating button widget
def create_expanded_button(description, button_style, width="100px"):
    return Button(
        description=description,
        button_style=button_style,
        layout=Layout(height="auto", width=width),
    )


# creates variable grid size
def create_grid(num_input, input_string, func):
    """
    - num_input = number of inputs in the function
    - input_string = a string of all the default values and input names that are passed in (ex: AB000011011)
    """
    num_row = (2**num_input) + 1
    num_col = num_input + 2
    grid = GridspecLayout(num_row, num_col, width=str(num_col * 117.5) + "px")

    # creates the default values for the different input combos of the truth table
    for i in range(0, num_row):
        for j in range(0, num_input):
            val = str(input_string[0])
            grid[i, j] = create_expanded_button(val, "info")
            input_string = input_string[1:]

    # creates the click to check button as well as the userinput section of the table
    for i in range(1, num_row):
        grid[i, num_input] = widgets.BoundedIntText(
            min=0, max=1, layout=Layout(height="auto", width="100px")
        )
        grid[0, num_col - 1] = create_expanded_button(
            "Click to Check", "warning", width="150px"
        )

    # creates the output section of the truth table to know if you're right or not
    for i in range(1, num_row):
        grid[i, num_col - 1] = create_expanded_button(" ", "warning", width="150px")

    grid[0, num_input] = create_expanded_button(func, "info")
    return grid


# checks if the user input is correct
def CheckAnswer(grid, num_inputs, input):
    for i in range (0,len(input)):
        if grid[i + 1, num_inputs].value == int(input[i]):
            grid[i + 1, num_inputs + 1].button_style = "Success"
            grid[i + 1, num_inputs + 1].description = "Correct!"
        else:
            grid[i + 1, num_inputs + 1].button_style = "Danger"
            grid[i + 1, num_inputs + 1].description = "Wrong! Submit again"


# AND gate truth table creation
grid1_1 = create_grid(2, 'AB00011011', 'F=AB')

# OR gate truth table creation
grid1_2 = create_grid(2, 'AB00011011', 'F=A+B')

# NOT gate truth table creation
grid1_3 = create_grid(1, "A01", '!A')

# NOR gate truth table creation
grid1_4 = create_grid(2, 'AB00011011', 'F=NOR')

# NAND gate truth table creatione
grid1_5 = create_grid(2, 'AB00011011', 'F=NAND')

# XOR gate truth table creation
grid1_6 = create_grid(2, 'AB00011011', 'F=A^B')

# XNOR gat truth table creatione
grid1_7 = create_grid(2, 'AB00011011', 'F=~(A^B')

# THIS CAN PROBABLY BE PLACED IN THE CREATE FUNCTION WITH F AS AN ARGUMENT ABCF000001010011100101110111


# LOOK INTO HAVING THE CHECK ANSWER BUTTON DO DIFFERENT THINGS
# DON'T LIKE THIS IMPLEMENTATION. Find a way to only need one 'on_button_clicked' function

# Process when clicking the "Check" button
# AND
def on_button_clicked(self):
    CheckAnswer(grid1_1, 2, '0001')


# OR
def on_button_clicked2(self):
    CheckAnswer(grid1_2, 2, '0111')


# NOT
def on_button_clicked3(self):
    CheckAnswer(grid1_3, 1, '10')


# NOR
def on_button_clicked4(self):
    CheckAnswer(grid1_4, 2, '1000')


# NAND
def on_button_clicked5(self):
    CheckAnswer(grid1_5, 2, '1110')


# XOR
def on_button_clicked6(self):
    CheckAnswer(grid1_6, 2, '0110')


# XNOR
def on_button_clicked7(self):
    CheckAnswer(grid1_7, 2, '1001')

In [ ]:
#@title Multiple Choice Creation
from IPython.display import IFrame, display, HTML
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
# function of creating button widget
def create_expanded_button(description, button_style, width='400px'):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width=width))

# function of creating grid layout
def create_grid(A,B,C,D):
    grid = GridspecLayout(5, 3, width='470px')
    grid[0,0]=create_expanded_button(A,'primary')
    grid[1,0]=create_expanded_button(B,'primary')
    grid[2,0]=create_expanded_button(C,'primary')
    grid[3,0]=create_expanded_button(D,'primary')  
    grid[4,0]=create_expanded_button('Submit','Danger')  
    return grid


def check_answermc4(self):
  """
  Used with .on_click to makesure the answer is correct
    self: This is a button. We have added 2 new attributes to it
      current_answer: This is the last clicked button and its description value stored as a str
      answer: This is the correct answer from the mcdict

  """
  if self.current_answer == 0:
    return
  if self.answer[0] == self.current_answer[0]:
    self.button_style = "Success"
    self.description = "Correct!"
  else:
    self.button_style = 'Danger'
    self.description = "Wrong, Try Again!"
"""
key: The question number
pair: This should be a list. indexes 0-3 are the choices. 4 is the answer. 5 is the question
answer: This must be the same str value as the choice
question: This is used in the header
"""
mc_dict = {

    "1": ["A. 0101","B. 01010", "C. 1010","D. Syntax Error", "A", "What is the value of A[5:2] where A = 9’b101010101?"],
    "2": ["A. 11010", "B. 10100", "C. 10111", "D. 01000", "B", "What is the value of A << 3 where A  = 5’b11101 ?"],
    "3": ["A. 10111", "B. 11101", "C. 101", "D. Syntax Error", "A", "What is the Value of {A,B} where A = 3’b101 and B = 2’11?"],
    "4": ["A. 1010", "B. 111000", "C. 101010", "D. Syntax Error", "C", "What is the value of {3{2’b10}}?"],
    "5": ["A.logic input [3:0] x", "B. logic [3:0] input x", "C. input logic x [3:0]", "D. input logic [3:0] x", "D", "Which signal is defined correctly?"  ],
    "6": ["A 1’b1", "B. 1’b0", "C. An Error will be thrown", "D. Undefined", "C", "What is the value of x from the code above?"],
    "7": ["A. 1’b1", "B. 1’b0", "C. 2’b11", "D. An Error will be thrown", "A", "What is the value of x from the above code?"] ,
    "8": [ "A. 4'b0001", "B. 4'b0011", "C. 4'b0111", "D. 4'b1001", "D", "What is the value of x in the above code?" ]
}
def multiple_choice4(key, dictionary):
  """
  Creates a multiple choice widget that has 4 options. 
  key: This is the key to the dictionary. It will be a str of digits. 
  dictionary: the dictionary used is mc_dict. Which houses the multiple choice answers.
  grid: The grid that has been set up and can be shown
  header_button: sets the header_button that houses the question description
  """
  answers = dictionary[key]
  question = answers[5]
  A = answers[0]
  B = answers[1]
  C = answers[2]
  D = answers[3]
  grid = create_grid(A,B,C,D)
  correct_answer = answers[4]
  setattr(grid[4,0], "answer", correct_answer)
  current_answer = 0
  setattr(grid[4,0], "current_answer", 0)
  grid[4,0].button_style = "Warning"
  def change_color(self):
    current_btn = int(self.layout.grid_area[-1])
    for i in range(0,4):
      grid[i,0].button_style = 'primary'
    grid[current_btn-1,0].button_style = 'info'
    grid[4,0].current_answer = self.description

  for i in range(0,4):
    grid[i,0].on_click(change_color)
  grid[4,0].on_click(check_answermc4)
  header_button = create_expanded_button(question, 'info', '465px')
  return grid, header_button


In [ ]:
#@title Verilator Install
!apt-get install verilator >/dev/null

In [ ]:
#@title Install PyGithub
!pip3 install PyGithub > /dev/null

In [ ]:
#@title vcd2dataframe
#@title vcd2dataframe
def vcd2dataframe(vcd_file):
  with open(vcd_file, "r") as f:
      # Remove Date, Program and  Timescale
      for i in range(20):
          line = f.readline()
          words = line.split()
          try:
            words[0]
          except:
            line = f.readline()
            words = line.split()
            continue
          if words[0] == "$date" or words[0] == "$timescale" or words[0] == "$version":
              for i in range(10):
                  line = f.readline()
                  words = line.split()
                  if words[0] == "$end" or words[-1] == "$end":
                      break
          else:
              break
          line = f.readline()
      # Create the dictionary of lists for inputs
      signals = {}
      line = f.readline()
      words = line.split()
      symbols = {}
      for i in range(50):
          try:
            words[0]
          except:
            line = f.readline()
            words = line.split()
            continue
          if words[0] == "$var":
              signals[words[4]] = []
              symbols[words[3]] = words[4]

          elif words[0] == "$enddefinitions":
              break
          line = f.readline()
          words = line.split()
      # print(signals)
      # print(symbols)
      k = 0

      value = 0
      for i in range(0, 13000):
          if not words:
              line = f.readline()
              words = line.split()
              continue
          elif words[0] == "$dumpvars":
              line = f.readline()
              words = line.split()
              for i in range(50):
                  if words[0] == "$end":
                      break
                  else:
                      lines = f.readline()
                      words = line.split()

          elif len(words) > 1:
              if words[0][0] == "b":
                  # print(line)
                  if words[0][1] == "x":
                      value = 0
                  else:
                      try:
                          value = int(words[0][1:],2)

                      except:
                          value = int(words[0][1],2)
                  symbol = words[1]
                  signals[symbols[symbol]].append(value)
          elif words[0][0] == "#" and words[0][0] != "0":
              k = k + 1
              for key in signals:
                  if len(signals[key]) < k:
                      try:
                          signals[key].append(signals[key][-1])
                      except:
                          signals[key].append(0)
          else:
              if words[0][0] == "x":
                  value = 0
              else:
                  value = int(words[0][0:-1])
              symbol = words[0][-1]
              signals[symbols[words[0][-1]]].append(value)
          line = f.readline()
          words = line.split()
      length_df = 0
      dataframe = signals
      for i in dataframe:
        if length_df < len(dataframe[i]):
          length_df = len(dataframe[i])
        # print(length_df)
      for i in dataframe:
        if len(dataframe[i]) < length_df:
          dataframe[i].append(dataframe[i][-1])
  return dataframe
    # print(signals)
    # Create a list of variables and their names (ignore types)
    # Create a key to translate the signals
    # Begin Cycling through the signals


In [ ]:
# @title Install Interactive Simulation
def altair_sim():
  dataframe = vcd2dataframe("/content/tmp_code/logs/vlt_dump.vcd")
  dataframe1 =dataframe
  df_max = 0
  for i in dataframe:
    if df_max < max(dataframe[i]):
      df_max = max(dataframe[i])
  import altair as alt
  import pandas as pd
  import numpy as np
  from vega_datasets import data
  length_df = 0
  for i in dataframe:
    if len(dataframe[i]) > length_df:
      length_df = len(dataframe[i])
  def make_waveform(dataframe):
    length_df = 0
    for i in dataframe:

      if len(dataframe[i]) > length_df:
        length_df = len(dataframe[i])

    source = pd.DataFrame( dataframe, index=pd.RangeIndex(length_df, name='x'))
    source = source.reset_index().melt('x', var_name='signals', value_name='y')
    scales = alt.selection_interval(bind='scales')

    multi_mouseover = alt.selection_multi(on='mouseover', toggle=True, empty='none')

    selection = alt.selection_multi(fields=['signals'], bind='legend')

    # Create a selection that chooses the nearest point & selects based on x-value
    nearest = alt.selection(type='multi', nearest=True, on='mouseover',
                            fields=['x'], empty='none')

    # The basic line
    line = alt.Chart(source).mark_line(interpolate='step-after').encode(
        x='x:Q',
        y='y:Q',
        color='signals:N'
    )

    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = alt.Chart(source).mark_point().encode(
        x='x:Q',
        opacity=alt.value(0),
    ).add_selection(
        nearest,
        scales
    )

    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(nearest, 'y:Q', alt.value(''))
    )

    # Draw a rule at the location of the selection
    rules = alt.Chart(source).mark_rule(color='black').encode(
        x='x:Q',
    ).transform_filter(
        nearest
    )

    # Put the five layers into a chart and bind the data
    waveform=alt.layer(
        line, selectors, points, rules, text
    ).encode(
        alt.X(scale=alt.Scale(domain=(0,length_df))),
        alt.Y(scale=alt.Scale(domain=(0,df_max+1))),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
    ).properties(
        width=600, height=400
    ).add_selection(
        selection
    )

    histogram = alt.Chart(source).mark_bar().encode(
      alt.X('y', 
          scale=alt.Scale(domain=(0,df_max + 1))),
      y='signals',
      color='signals:N',

    ).transform_filter(nearest).properties(
        width = 600, height = 200
    )
    text = histogram.mark_text(
        align = 'left',
        baseline = 'middle',
        dx = 3
    ).encode(
        text = 'y:Q'
    )
    histogram = histogram + text  
    return waveform, histogram

  waveform1, histogram1 = make_waveform(dataframe1)
  return waveform1, histogram1


In [ ]:
#@title Install Extension
!pip install --upgrade git+https://github.com/byuccl/nb_js_diagrammers.git

In [ ]:
#@title Load Extension
%load_ext nb_js_diagrammers

In [ ]:
#@title dataframe to wavedrom
def df2wd(filename):
    dataframe = vcd2dataframe("/content/tmp_code/logs/vlt_dump.vcd")
    string_list = []
    name_list = []
    starting_value = 4
    for i in dataframe:
        value = ""
        value_str = ""
        last_value = ""
        for j in dataframe[i]:
            value_list = dataframe[i]
            if last_value == str(j):
                value = "."
            else:
                if int(j) > 2:
                    x = starting_value
                    starting_value += 1
                    if starting_value >= 9:
                        starting_value = 4
                    value = str(x)
                else:
                    value = str(j)
                last_value = str(j)
            value_str = value_str + value
        string_list.append(value_str)
        name_list.append(i)

    wavedrom_format = """{ signal : ["""
    for i in range(len(string_list)):
        if name_list[i] == "clk":
            new_line = (
                """{ name: "clk",  wave: "p"""
                + "." * (-1 + len(value_str))
                + """\", period: 2 },"""
                + f"\n"
            )
        elif max(dataframe[name_list[i]]) > 1:
            int_list = dataframe[name_list[i]]
            new_string_list = string_list[i]
            int_string = ""
            current_value = ""
            last_value = ""
            # Fix String List
            new_string_list =  new_string_list.replace("0", "2")
            new_string_list = new_string_list.replace("1", "3")
            # Create a Data Set
            for j in int_list:
                current_value = j
                if current_value == last_value:
                    pass
                else:
                    int_string += str(current_value) + " "
                last_value = current_value
            new_line = (
                '{ name: "'
                + name_list[i]
                + """\",  wave: \""""
                + new_string_list
                + """\", data:\" """
                + int_string
                + """ \" },"""
                + f"\n"
            )
        else:
            new_line = (
                """{ name: \""""
                + name_list[i]
                + """ \",  wave: \""""
                + string_list[i]
                + """\" },"""
                + f"\n"
            )
        wavedrom_format += new_line
    wavedrom_format += """
    ]}
"""
    filename = "/content/tmp_code/" + filename + ".txt"
    with open(filename, "w") as f:
        f.write(wavedrom_format)



## How computers do math


### How to represent Negative Binary Numbers

What do we do if we want to represent a negative number in binary? Well a simple way would be to have the most significant bit assigned to represent if it was negative or positive. 

This is called a sign-magnitude number. A `1` is negative and a `0` is positive. So `1001` would be `-1` and `0001` would be `1`. 

There is one problem here.

With 4 bits we have 16 unique combinations and each can represent a unique value. The problem with this method is that there are two ways to represent 0. `1000` and `0000` are both ways to represent 0, and aren't unique. This means that only 15 values can be represented.<br>This may seem like a small thing, but when every bit matters, it can add up.
<br><br>
This is where two’s complement solves this problem. Instead of having the MSB (most significant bit) be used to represent negative or positive, the MSB is equal to its regular value but negative.
So `1000` is equal to `-8`. So `1111` is equal to `-1`. Instead of only being able to represent `-7` to `7` it can now represent `-8` to `7`.  


In [ ]:
#@title ㅤ
display(HTML("""
<style>
div.warn {    
    color: #356A89;
    background-color: #D4EAF7;
    border-left: 5px solid #3C82E3;
    padding: 0.5em;
    width: 900px
    }
 </style>
<div class=warn>
NOTE: Enter values in Decimal unless otherwise specified.
</div>"""))

In [ ]:
#@title 
frq_1[0,2].on_click(check_frq_1)
header_button = create_expanded_button('Sign-magnitude 1', 'primary', '600px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_1])

In [ ]:
#@title
frq_2[0,2].on_click(check_frq_2)
header_button = create_expanded_button('Sign-magnitude 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_2])

In [ ]:
#@title
frq_3[0,2].on_click(check_frq_3)
header_button = create_expanded_button('Two\'s compliment 1', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_3])

In [ ]:
#@title
frq_4[0,2].on_click(check_frq_4)
header_button = create_expanded_button('Two\'s compliment 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_4])

###Inverting Two's Complement

If it was just a sign-magnitude number, we could flip the most significant bit to invert it, but doing that with two’s complement doesn’t quite work. `1001` which is  `-7` would be `0001` which is `1`. So instead we will flip every single bit. `1001` becomes `0110` respectively `-7` and `6` which is close. So all we have to do is add `1` resulting in `0111` which is equal to `7`.   

So, to flip the sign of a two’s complement, all you have to do is flip every single bit and add one.


In [ ]:
#@title
frq_5[0,2].on_click(check_frq_5)
header_button = create_expanded_button('Inverting 1', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_5])

In [ ]:
#@title
frq_6[0,2].on_click(check_frq_6)
header_button = create_expanded_button('Inverting 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_6])


###Adding Unsigned Binary Numbers

Adding binary numbers is very similar to basic elementary addition. You add column by column, and you carry over a one if a column adds up to 2 or 3.   
Example 1:  

&nbsp;&nbsp;&nbsp;0110 &nbsp;&nbsp;&nbsp;&nbsp;(6<sub>10</sub>)  
<u>+ 0001</u> &nbsp;&nbsp;&nbsp;&nbsp;(1<sub>10</sub>)  
&nbsp;&nbsp;&nbsp;0111    &nbsp;&nbsp;&nbsp;&nbsp;(7<sub>10</sub>)  

_____  
\
Example 2:  
<sub>&nbsp;&nbsp;&nbsp;${\scriptsize 1}$&nbsp;${\scriptsize 1}$ </sub>  
&nbsp;&nbsp;&nbsp;0110 &nbsp;&nbsp;&nbsp;&nbsp;(6<sub>10</sub>)  
<u>+ 0010 </u> &nbsp;&nbsp;&nbsp;&nbsp;(2<sub>10</sub>)  
&nbsp;&nbsp;&nbsp;1000    &nbsp;&nbsp;&nbsp;&nbsp;(8<sub>10</sub>)  
 





In [ ]:
#@title
frq_7[0,2].on_click(check_frq_7)
header_button = create_expanded_button('Unsigned Add 1', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_7])

In [ ]:
#@title
frq_8[0,2].on_click(check_frq_8)
header_button = create_expanded_button('Unsigned Add 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_8])

In [ ]:
#@title
frq_9[0,2].on_click(check_frq_9)
header_button = create_expanded_button('Unsigned Add 3', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_9])

### Overflow

But there is a problem, when adding binary numbers in hardware, we have a set number of wires. This means we only have so many bits assigned to each signal. 4 bits can only hold a value up to `15` if its unsigned. So what happens if we add `14` and `14`?
This is called overflow  


  
<sub>&nbsp;${\scriptsize 1}$&nbsp;${\scriptsize 1}$&nbsp;${\scriptsize 1}$ </sub>  
&nbsp;&nbsp;&nbsp;1110 &nbsp;&nbsp;&nbsp;&nbsp;(14<sub>10</sub>)  
<u>+ 1110 </u> &nbsp;&nbsp;&nbsp;&nbsp;(14<sub>10</sub>)  
&nbsp;11100    &nbsp;&nbsp;&nbsp;&nbsp;(28<sub>10</sub>)  


There is a simple problem, this number is actually 5 bits long and not 4. So when each bit is tied to a wire, and we have 4 wires, where does the 5th bit go? Often it is discarded and we should just be more careful to avoid overflow.
This can cause an error in the program as `11100` (28) would become `1100` (12). The output is not what is expected and can lead to errors and bugs. There are no automatic checks for overflow, so programmers must be aware of this.

Another problem is that when dealing with Two's Complement, there is no guarantee that even if we could use this 5 bit number, that it would be correct. That is unless we sign extend. 


In [ ]:
#@title
frq_10[0,2].on_click(check_frq_10)
header_button = create_expanded_button('Overflow 1', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_10])

In [ ]:
#@title
frq_11[0,2].on_click(check_frq_11)
header_button = create_expanded_button('Overflow 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_11])

###Adding Two's Complement

Adding two’s compliment is very similar to adding unsigned binary.

<sub>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;${\scriptsize 1}$ </sub>  
&nbsp;&nbsp;&nbsp;1001 &nbsp;&nbsp;&nbsp;&nbsp;(-7<sub>10</sub>)  
<u>+ 0101 </u> &nbsp;&nbsp;&nbsp;&nbsp;(5<sub>10</sub>)  
&nbsp;&nbsp;&nbsp;1110 &nbsp;&nbsp;&nbsp;&nbsp;(-2<sub>10</sub>) 

However, when there is overflow things get a little weird. Sometimes you can just append the final carry output to the left side, but sometimes it doesn't work.  
<sub>&nbsp;${\scriptsize 1}$&nbsp;&nbsp;&nbsp;${\scriptsize 1}$ ${\scriptsize 1}$</sub>  
&nbsp;&nbsp;&nbsp;1001 &nbsp;&nbsp;&nbsp;&nbsp;(-7<sub>10</sub>)  
<u>+ 1011 </u> &nbsp;&nbsp;&nbsp;&nbsp;(-5<sub>10</sub>)  
&nbsp;10100 &nbsp;&nbsp;&nbsp;&nbsp;(-12<sub>10</sub>)  
_____  

<sub>&nbsp;${\scriptsize 1}$&nbsp;${\scriptsize 1}$ &nbsp;&nbsp;${\scriptsize 1}$</sub>  
&nbsp;&nbsp;&nbsp;1101 &nbsp;&nbsp;&nbsp;&nbsp;(-3<sub>10</sub>)  
<u>+ 0101 </u> &nbsp;&nbsp;&nbsp;&nbsp;(5<sub>10</sub>)  
&nbsp;10010 &nbsp;&nbsp;&nbsp;&nbsp;(-14<sub>10</sub>)  

A solution is to sign extend before doing the computation. This means if the MSB is a 0, put an extra 0 at the start, and if the MSB is a 1 put an extra 1 at the start. This does not change the value of the numbers.

1101 -> 11101  
0101 -> 00101 

So now we add:

<sub>&nbsp;${\scriptsize 1}$&nbsp;${\scriptsize 1}$&nbsp;${\scriptsize 1}$ &nbsp;&nbsp;${\scriptsize 1}$</sub>  
&nbsp;&nbsp;&nbsp;11101 &nbsp;&nbsp;&nbsp;&nbsp;(-3<sub>10</sub>)  
<u>+ 00101 </u> &nbsp;&nbsp;&nbsp;&nbsp;(5<sub>10</sub>)  
&nbsp;&nbsp;&nbsp;00010 &nbsp;&nbsp;&nbsp;&nbsp;(2<sub>10</sub>)  

You throw away the final carry out bit and are left with a 5 bit number. Now we see that -3 + 5 actually does equal 2. This method will always work, but it requires extra logic to sign extend and uses one more bit when implemented in hardware. Sometimes it is better to just detect overflow so you know when to ignore erroneous results.  

This works because sign-extending a Two's Complement number doesn't change its value. For example, `101` is equal to `11111101` and `0000001` is equal to `01`.




In [ ]:
#@title
frq_14[0,2].on_click(check_frq_14)
header_button = create_expanded_button("Adding 2's Complement 1", 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_14])

In [ ]:
#@title
frq_15[0,2].on_click(check_frq_15)
header_button = create_expanded_button("Adding 2's Complement 2", 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_15])

In [ ]:
#@title
frq_16[0,2].on_click(check_frq_16)
header_button = create_expanded_button("Adding 2's Complement 3", 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_16])

###Sign Extension

Sign Extension:  
There is a simple solution to overflow, just make your numbers wider. This can also allow us to add two numbers of different bit length, since the MSB needs to line up, for example a 3 bit and a 4 bit wide number.  
To do this when using Two's complement, just repeat the MSB.  
 For example, to make `0110` wider just add `0`s to the front: `00110`, `0000110` etc. This does not change the value. This also works for negative. `111` becomes `1111` becomes `1111111` etc. All these still equal `-1`.  




In [ ]:
#@title
frq_12[0,2].on_click(check_frq_12)
header_button = create_expanded_button('Sign Extension 1', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_12])

In [ ]:
#@title
frq_13[0,2].on_click(check_frq_13)
header_button = create_expanded_button('Sign Extension 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_13])

###Subtracting Two's Complement

There are two ways to subtract two’s complement numbers.  
 One is to just invert the second value, then add them together. (This is the most practical for hardware implementation)     
 The other is to subtract using borrows instead of carry overs (just like you did in elementary school).

In [ ]:
#@title
frq_17[0,2].on_click(check_frq_17)
header_button = create_expanded_button("Subtracting 2's Complement 1", 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_17])

In [ ]:
#@title
frq_18[0,2].on_click(check_frq_18)
header_button = create_expanded_button("Subtracting 2's Complement 2", 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_18])

###Gate Level Adder

To know how a computer can add `6` and `8` to get `14` we must first understand how a computer can add a just 2 bits together.
Our circuit will have just 2 inputs A and B which we will add together.
There are three possible results `0`, `1` or `2` (which will cause a carry over and the output will be `0`).  
So if A or B, but not both, are `0` then the output is `1`. That's an XOR gate. 
And if A and B are `1` we get carried over, that's an AND gate.
A simple adder would look like this. 

! [Picture] (https://raw.githubusercontent.com/byuccl/digital_design_colab/main/Arithmetic/media/adder.png)

To make a circuit that can add multi-bit numbers, you just chain a bunch of adders together. These are called ripple adders.
But now what if this isn’t the LSB we are adding. What if it's right in the middle? It now needs to be able to accept a carry over from a different order.
Now there are 4 possible results `0`, `1`, `2` or `3`. In this case, `3` will be a `1` with a carry over. 
If either 1 or 3 of the inputs are `1` we get a `1` as the output. So it's still an XOR, but now with 3 inputs.
Now, if 2 or more are `1` then we get carry over. If either A and B, B and Cin or A and Cin, are `1` then the carry over is `1`. 
This can be represented by having 3 AND gates (AB, A Cin, B Cin) all feeding an OR gate. Like so.

In [ ]:
#@title
frq_19[0,2].on_click(check_frq_19)
header_button = create_expanded_button("Full Adder", 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_19])

In [ ]:
#@title
frq_20[0,2].on_click(check_frq_20)
header_button = create_expanded_button("Full Adder", 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_20])

###Ripple-Adder

The Ripple-Adder is made of multiple full adders. Each bit is its own full adder that computes one bit. 

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/main/Arithmetic/media/add8.png)

For example,  
To compute `001`+`011`:    
The first full adder will have A=`1` and B=`1`. The output will be `0` and there will be a carry out.  
The second full adder will now have A=`0`, B=`1` and Cin= `1` (Cin is the carry out from the last full adder). The output will be `0` and there will be a carry out.   
For the third full adder A=`0` and B=`0` but Cin=`1` so the output is `1` with no carry out.   
This gives us a final answer of `100` or `4`.


##The Lab

###Full Adder

The first building block of your Circuit will be the full adder.  

| Module Name: |Full_Add|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|a 	|Input 	|1| 	‘a’ operand input|
|b 	|Input 	|1 |	‘b’ operand input|
|cin |	Input| 	1| 	Carry in|
|s 	|Output 	|1| 	Sum output|
|co| 	Output 	|1| 	Carry out output|

It will add together `a` and `b` and a carry over from another module. (If it is the first bit the `Cin` will be `0`)   
It will then give `s` is the sum of the inputs. If there is a carry out, `co` should be `1`.


In [ ]:
#@title Launch Simulation Workspace 
createSimulationWorkSpace("tmp_code/full_add")


In [ ]:
#@title Verilator TestBench
# !verilator --cc full_add 
!verilator -Wall --trace -cc full_add.sv --exe tb_full_add.cpp
!make -C obj_dir -f Vfull_add.mk Vfull_add > /dev/null
!./obj_dir/Vfull_add

In [ ]:
#@title Create WaveDrom
df2wd("full_add")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/full_add.txt
---

###What is a module?  



Instead of copying and pasting a Full-Adder over and over, we can instantiate it again as a module.  
This is the same concept as making a function call in python or C++. We are just reusing code we have already written.  
It follows the following format:  
`module_name module_title (.first_signal(first_connection), .second_signal(second_connection));`  
The first line will be whatever you have called your module. In this case it will be `Full_Add`.  
Then you must give each full adder a name. This part isn't important, but if its name has meaning it will be easier to understand your code. So, label each adder depending on its position.
Now you must define which internal signals go to which external signals.  
For example, if we want to add 8 bits together, for the first full adder, a should be the first bit of an a bit input. The `Cin` should be `0`. `s` should be the first bit of an output and `co` should be neither an input nor an output, but should instead be an intermediate signal that is used for the second bit.


###Creating an 8-bit adder


To create an 8-bit adder, you will need 8 one bit adders all linked together.

| Module Name: |8Add|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|a 	|Input 	|8| 	‘a’ operand input|
|b 	|Input 	|8 |	‘b’ operand input|
|cin |	Input| 1	| 	Carry in|
|s 	|Output 	|8| 	Sum output|
|co| 	Output 	|1| 	Carry out output of the last bit|


You can use various internal signals for `Cin` and `Cout`.  
This module should have 8 instances of your Full_Add module.

In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/add_8")

In [ ]:
#@title Verilator TestBench
!verilator --cc add_8 
!verilator -Wall --trace -cc add_8.sv --exe tb_add_8.cpp
!make -C obj_dir -f Vadd_8.mk Vadd_8 > /dev/null
!./obj_dir/Vadd_8

### Top level module

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/main/Arithmetic/media/lab_top.png)

You probably noticed that there is no `led` or `sw` signal yet. This will be contained in a top level module. You will instantiate 8Add and feed into it all the necessary signals.

| Module Name: |arithemtic_top|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|sw 	|Input 	|16| Switches (`sw[15:8]` = `b`, `sw[7:0]` = `a` operand)	|
|btnc |	Input| 1	| Subtract when pressed|
|led	|Output 	|9| 	LEDs (`led[8]`= overflow, `led[7:0]` = sum)|  

While `btnc` is high, `b` should be inverted to subtract `b` from `a`.  
Led[8] should be high if overflow occurs.



### Overflow Detection  




###How to find of Overflow Occured

While using two's complement we can easily check for overflow. 
If the MSB's are the same, but the output is different then overflow has occurred. For example, `1111`+`1000`, both MSB are `1`. The output would be `0111`. Since `0`, the MSB, is different than `1`, we can tell that overflow has occurred.    


In other words, if you add two negative numbers and get a positive number, overflow has occurred. If you add two negative numbers and the output is positive, overflow has occurred.



###How does subtraction work?
As we explained earlier, there are two ways to do subtraction with two's complement, either using borrows like in elementary school, or inverting the second operator.  
Unfortunately, our computers never went to elementary school, so they will have to do the second option.  
Luckily, inverting `b` is quite simple.  
But you must also add `1` when inverting. Also, we have a carry in signal for our first bit, which acts exactly like adding `1`.    
This means that we can write our code such that if that carry in is high, then subtraction should occur.

In [ ]:
#@title
createSimulationWorkSpace("tmp_code/arithmetic_top")

In [ ]:
#@title Create WaveDrom
df2wd("top")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/top.txt
---

In [ ]:
#@title Generate an XDC File
%%bash -c 'cat > /content/tmp_code/xdc.xdc'
###################################################################
###################################################################
## Buttons
set_property -dict { PACKAGE_PIN U18   IOSTANDARD LVCMOS33 } [get_ports { btnc }];
# set_property -dict { PACKAGE_PIN T18   IOSTANDARD LVCMOS33 } [get_ports { btnu }];
# set_property -dict { PACKAGE_PIN W19   IOSTANDARD LVCMOS33 } [get_ports { btnl }];
# set_property -dict { PACKAGE_PIN T17   IOSTANDARD LVCMOS33 } [get_ports { btnr }];
# set_property -dict { PACKAGE_PIN U17   IOSTANDARD LVCMOS33 } [get_ports { btnd }];

## Switches
set_property -dict { PACKAGE_PIN V17   IOSTANDARD LVCMOS33 } [get_ports { sw[0] }];
set_property -dict { PACKAGE_PIN V16   IOSTANDARD LVCMOS33 } [get_ports { sw[1] }];
set_property -dict { PACKAGE_PIN W16   IOSTANDARD LVCMOS33 } [get_ports { sw[2] }];
set_property -dict { PACKAGE_PIN W17   IOSTANDARD LVCMOS33 } [get_ports { sw[3] }];
set_property -dict { PACKAGE_PIN W15   IOSTANDARD LVCMOS33 } [get_ports { sw[4] }];
set_property -dict { PACKAGE_PIN V15   IOSTANDARD LVCMOS33 } [get_ports { sw[5] }];
set_property -dict { PACKAGE_PIN W14   IOSTANDARD LVCMOS33 } [get_ports { sw[6] }];
set_property -dict { PACKAGE_PIN W13   IOSTANDARD LVCMOS33 } [get_ports { sw[7] }];
set_property -dict { PACKAGE_PIN V2    IOSTANDARD LVCMOS33 } [get_ports { sw[8] }];
set_property -dict { PACKAGE_PIN T3    IOSTANDARD LVCMOS33 } [get_ports { sw[9] }];
set_property -dict { PACKAGE_PIN T2    IOSTANDARD LVCMOS33 } [get_ports { sw[10] }];
set_property -dict { PACKAGE_PIN R3    IOSTANDARD LVCMOS33 } [get_ports { sw[11] }];
set_property -dict { PACKAGE_PIN W2    IOSTANDARD LVCMOS33 } [get_ports { sw[12] }];
set_property -dict { PACKAGE_PIN U1    IOSTANDARD LVCMOS33 } [get_ports { sw[13] }];
set_property -dict { PACKAGE_PIN T1    IOSTANDARD LVCMOS33 } [get_ports { sw[14] }];
set_property -dict { PACKAGE_PIN R2    IOSTANDARD LVCMOS33 } [get_ports { sw[15] }];

## LEDs
set_property -dict { PACKAGE_PIN U16   IOSTANDARD LVCMOS33 } [get_ports { led[0] }];
set_property -dict { PACKAGE_PIN E19   IOSTANDARD LVCMOS33 } [get_ports { led[1] }];
set_property -dict { PACKAGE_PIN U19   IOSTANDARD LVCMOS33 } [get_ports { led[2] }];
set_property -dict { PACKAGE_PIN V19   IOSTANDARD LVCMOS33 } [get_ports { led[3] }];
set_property -dict { PACKAGE_PIN W18   IOSTANDARD LVCMOS33 } [get_ports { led[4] }];
set_property -dict { PACKAGE_PIN U15   IOSTANDARD LVCMOS33 } [get_ports { led[5] }];
set_property -dict { PACKAGE_PIN U14   IOSTANDARD LVCMOS33 } [get_ports { led[6] }];
set_property -dict { PACKAGE_PIN V14   IOSTANDARD LVCMOS33 } [get_ports { led[7] }];
set_property -dict { PACKAGE_PIN V13   IOSTANDARD LVCMOS33 } [get_ports { led[8] }];
set_property -dict { PACKAGE_PIN V3    IOSTANDARD LVCMOS33 } [get_ports { led[9] }];
set_property -dict { PACKAGE_PIN W3    IOSTANDARD LVCMOS33 } [get_ports { led[10] }];
set_property -dict { PACKAGE_PIN U3    IOSTANDARD LVCMOS33 } [get_ports { led[11] }];
set_property -dict { PACKAGE_PIN P3    IOSTANDARD LVCMOS33 } [get_ports { led[12] }];
set_property -dict { PACKAGE_PIN N3    IOSTANDARD LVCMOS33 } [get_ports { led[13] }];
set_property -dict { PACKAGE_PIN P1    IOSTANDARD LVCMOS33 } [get_ports { led[14] }];
set_property -dict { PACKAGE_PIN L1    IOSTANDARD LVCMOS33 } [get_ports { led[15] }];



## Compiling with the F4PGA Toolchain


### Installing the Toolchain

In [ ]:
!apt install -y git wget xz-utils

In [ ]:
%%bash
git clone https://github.com/chipsalliance/f4pga-examples
cd f4pga-examples
time wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O conda_installer.sh


In [ ]:
#Creates the Conda Environment
%%bash
cd f4pga-examples
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
time bash conda_installer.sh -u -b -p $F4PGA_INSTALL_DIR/$FPGA_FAM/conda;
time source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
time conda env create -f $FPGA_FAM/environment.yml

In [ ]:
#Download F4PGA Arch Defs
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
mkdir -p $F4PGA_INSTALL_DIR/xc7/install

F4PGA_TIMESTAMP='20220907-210059'
F4PGA_HASH='66a976d'
export F4PGA_PACKAGES='install-xc7 xc7a50t_test'

for PKG in $F4PGA_PACKAGES; do
  wget -qO- https://storage.googleapis.com/symbiflow-arch-defs/artifacts/prod/foss-fpga-tools/symbiflow-arch-defs/continuous/install/${F4PGA_TIMESTAMP}/symbiflow-arch-defs-${PKG}-${F4PGA_HASH}.tar.xz | tar -xJC $F4PGA_INSTALL_DIR/${FPGA_FAM}
done

### Compiling with the Toolchain

In [ ]:
#This creates the Makefile
with open("/content/tmp_code/Makefile", "w") as f:
  f.write("""current_dir := ${CURDIR}
TARGET := basys3

TOP := arithmetic_top

XDC := ${current_dir}/*.xdc

SOURCES := $(wildcard ${current_dir}/*.v ${current_dir}/*.sv)

include /content/f4pga-examples/common/common.mk
""")

In [ ]:
#Make the project
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
export TARGET=basys3
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
mkdir -p /content/Arithemtic
cd /content/tmp_code
cp xdc.xdc /content/Arithemtic/xdc.xdc
cp Makefile /content/Arithemtic/Makefile
cp *.sv /content/Arithmetic/arithetmic_top.sv
conda activate xc7
cd /content/Arithmetic
conda activate xc7
make 2> ./error.log 1>./compile.log
cp ./build/basys/*.bit ./arithemtic_top.bit

## Testing it on the board

You can use this configuration file and the program openOCD to download the bitstream to your board. You will need to download this file and the bitstream to your local machine.

In [ ]:
#Create File for openocd
with open("/content/arithmetic_top.cfg", "w") as f:
  f.write("""interface ftdi
ftdi_device_desc "Digilent USB Device"
ftdi_vid_pid 0x0403 0x6010
# channel 1 does not have any functionality
ftdi_channel 0
# just TCK TDI TDO TMS, no reset
ftdi_layout_init 0x0088 0x008b
reset_config none
adapter_khz 10000

source [find cpld/xilinx-xc7.cfg]
source [find cpld/jtagspi.cfg]
init

puts [irscan xc7.tap 0x09]

set xc7a35t "0362D093"
set xc7a100t "13631093"
set code [drscan xc7.tap 32 0]  
puts $code

if { $code == $xc7a35t} {
    puts "The board has an xc7a35t"
}

if { $code == $xc7a100t} {
    puts "The board has an xc7a100t"
}

puts "Programming..."
pld load 0 arithemtic_top.bit
exit"""
)

The following code will download a zip file with the configuration file and bitstream. This will only download if you are using Chrome. If you are using something else, click the file icon on the left, and click the zip file in the content folder to download it.

In [ ]:
#@title Create Zip
%%bash
cd /content
mv Arithmetic/build/basys3/*.bit ./
zip arithemtic.zip *.bit arithmetic_top.cfg